## Setup: Install Dependencies

In [1]:
# Install required packages if needed
import sys
import subprocess

packages = [
    'rasterio',
    'numpy',
    'matplotlib',
    'opencv-python',
    'scipy',
    'utm',
    'pillow'
]

for package in packages:
    try:
        __import__(package.replace('-', '_'))
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])

print("✓ Dependencies installed")

Installing opencv-python...
Defaulting to user installation because normal site-packages is not writeable
Installing pillow...
Defaulting to user installation because normal site-packages is not writeable
✓ Dependencies installed


## Step 1: Setup - Imports and Paths

In [2]:
import numpy as np
import rasterio
from rasterio.transform import xy
from rasterio.warp import transform as transform_coords
from pathlib import Path
import cv2
import matplotlib.pyplot as plt
from scipy import ndimage
import json
import csv
import utm
from typing import Dict, List, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

# Setup paths
data_dir = Path("/Users/mauriciohessflores/Documents/Code/Data/New Westminster Oct _25")
output_dir = Path("outputs")

# Input files
basemap_path = data_dir / "Michael_RTK_orthos" / "TestsiteNewWest_Spexigeo_RTK.tiff"
gcp_csv_path = data_dir / "25-3288-CONTROL-NAD83-UTM10N-EGM2008.csv"
ortho_no_gcps_path = output_dir / "orthomosaics" / "orthomosaic_no_gcps.tif"
ortho_with_gcps_path = output_dir / "orthomosaics" / "orthomosaic_with_gcps.tif"

# Output directories
gcp_matching_dir = output_dir / "gcp_matching"
gcp_matching_dir.mkdir(parents=True, exist_ok=True)

patches_dir = gcp_matching_dir / "patches"
patches_dir.mkdir(exist_ok=True)

matches_dir = gcp_matching_dir / "matches"
matches_dir.mkdir(exist_ok=True)

registered_dir = gcp_matching_dir / "registered"
registered_dir.mkdir(exist_ok=True)

print(f"✓ Output directory: {gcp_matching_dir}")
print(f"  - Patches: {patches_dir}")
print(f"  - Matches: {matches_dir}")
print(f"  - Registered: {registered_dir}")

✓ Output directory: outputs/gcp_matching
  - Patches: outputs/gcp_matching/patches
  - Matches: outputs/gcp_matching/matches
  - Registered: outputs/gcp_matching/registered


In [3]:
# Load GCPs from UTM CSV file
def load_gcps_from_csv(csv_path: Path) -> List[Dict]:
    """
    Load GCPs from UTM CSV file and convert to WGS84.
    
    Expected format: ID, Northing, Easting, Elevation, Name
    """
    import csv
    
    gcps = []
    
    with open(csv_path, 'r') as f:
        # Try to detect if there's a header
        first_line = f.readline().strip()
        f.seek(0)  # Reset to beginning
        
        # Check if first line is numeric (no header)
        try:
            float(first_line.split(',')[0])
            has_header = False
        except (ValueError, IndexError):
            has_header = True
        
        reader = csv.reader(f) if not has_header else csv.DictReader(f)
        
        for row_idx, row in enumerate(reader):
            try:
                if has_header:
                    # Try to find columns
                    northing = float(row.get('Northing', row.get('northing', row.get('Y', 0))))
                    easting = float(row.get('Easting', row.get('easting', row.get('X', 0))))
                    gcp_id = row.get('Name', row.get('name', row.get('ID', row.get('id', f"GCP_{row_idx+1}"))))
                else:
                    # Positional format: ID, Northing, Easting, Elevation, Name
                    if len(row) < 3:
                        continue
                    gcp_id = row[0].strip() if row[0] else f"GCP_{row_idx+1}"
                    northing = float(row[1])  # Column 1 = Northing
                    easting = float(row[2])   # Column 2 = Easting
                
                # Convert UTM to WGS84 (UTM Zone 10N)
                lat, lon = utm.to_latlon(easting, northing, 10, 'N')
                
                gcps.append({
                    'id': gcp_id,
                    'lat': lat,
                    'lon': lon,
                    'x_utm': easting,
                    'y_utm': northing
                })
            except (ValueError, IndexError) as e:
                print(f"⚠️  Skipping row {row_idx+1}: {e}")
                continue
    
    return gcps

## Step 2: Load GCPs from CSV and Convert to WGS84

In [4]:
# Load GCPs - try existing WGS84 files first, otherwise parse CSV
import json

# Check for existing WGS84 GCP files from ground control comparison
gcps_wgs84_geojson = output_dir / "ground_control_comparison" / "gcps_wgs84.geojson"
gcps_wgs84_csv = output_dir / "ground_control_comparison" / "gcps_wgs84.csv"

gcps = []

# Try GeoJSON first (preferred)
if gcps_wgs84_geojson.exists():
    print(f"Loading GCPs from GeoJSON: {gcps_wgs84_geojson}")
    with open(gcps_wgs84_geojson, 'r') as f:
        geojson_data = json.load(f)
    
    if 'features' in geojson_data:
        for feature in geojson_data['features']:
            props = feature.get('properties', {})
            geom = feature.get('geometry', {})
            
            if geom.get('type') == 'Point':
                coords = geom.get('coordinates', [])
                if len(coords) >= 2:
                    lon, lat = coords[0], coords[1]
                    
                    # Get UTM coordinates from properties or convert
                    x_utm = props.get('x_utm')
                    y_utm = props.get('y_utm')
                    
                    if x_utm is None or y_utm is None:
                        # Convert WGS84 to UTM
                        import utm
                        x_utm, y_utm, zone_num, zone_letter = utm.from_latlon(lat, lon)
                    
                    gcps.append({
                        'id': props.get('id', props.get('name', f"GCP_{len(gcps)+1}")),
                        'lat': lat,
                        'lon': lon,
                        'x_utm': float(x_utm),
                        'y_utm': float(y_utm)
                    })
    
    print(f"✓ Loaded {len(gcps)} GCPs from GeoJSON")

# Try CSV if GeoJSON not found
elif gcps_wgs84_csv.exists():
    print(f"Loading GCPs from WGS84 CSV: {gcps_wgs84_csv}")
    with open(gcps_wgs84_csv, 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            try:
                lat = float(row.get('lat', row.get('latitude', 0)))
                lon = float(row.get('lon', row.get('longitude', row.get('lon', 0))))
                gcp_id = row.get('id', row.get('name', row.get('label', f"GCP_{len(gcps)+1}")))
                
                # Get UTM from row or convert
                x_utm_str = row.get('x_utm', '')
                y_utm_str = row.get('y_utm', '')
                
                if x_utm_str and y_utm_str:
                    x_utm = float(x_utm_str)
                    y_utm = float(y_utm_str)
                else:
                    import utm
                    x_utm, y_utm, zone_num, zone_letter = utm.from_latlon(lat, lon)
                
                gcps.append({
                    'id': gcp_id,
                    'lat': lat,
                    'lon': lon,
                    'x_utm': x_utm,
                    'y_utm': y_utm
                })
            except (ValueError, KeyError) as e:
                print(f"⚠️  Skipping row: {e}")
                continue
    
    print(f"✓ Loaded {len(gcps)} GCPs from WGS84 CSV")

# Fallback to parsing UTM CSV
if len(gcps) == 0:
    print(f"\nNo WGS84 GCP files found, parsing UTM CSV: {gcp_csv_path}")
    gcps = load_gcps_from_csv(gcp_csv_path)
    print(f"✓ Loaded {len(gcps)} GCPs from UTM CSV")

if len(gcps) > 0:
    print(f"\nFirst few GCPs:")
    for gcp in gcps[:3]:
        print(f"  {gcp['id']}: UTM=({gcp['x_utm']:.2f}, {gcp['y_utm']:.2f}), WGS84=({gcp['lat']:.6f}, {gcp['lon']:.6f})")
else:
    print(f"⚠️  No GCPs loaded!")
    print(f"   Checked:")
    print(f"   - {gcps_wgs84_geojson}")
    print(f"   - {gcps_wgs84_csv}")
    print(f"   - {gcp_csv_path}")


Loading GCPs from GeoJSON: outputs/ground_control_comparison/gcps_wgs84.geojson
✓ Loaded 23 GCPs from GeoJSON

First few GCPs:
  GCP1: UTM=(506914.12, 5450945.53), WGS84=(49.211262, -122.905068)
  GCP2: UTM=(506657.79, 5450730.01), WGS84=(49.209326, -122.908591)
  GCP3: UTM=(506577.77, 5450480.01), WGS84=(49.207078, -122.909694)


## Step 3: Convert GCPs to Pixel Coordinates in Basemap

In [5]:
# Convert GCPs (UTM) to pixel coordinates in basemap
def gcp_to_pixel_coords_from_utm(gcp_x_utm: float, gcp_y_utm: float, raster_path: Path) -> Optional[Tuple[int, int]]:
    """
    Convert GCP UTM coordinates to pixel coordinates in raster.
    
    Args:
        gcp_x_utm: UTM Easting (EPSG:32610)
        gcp_y_utm: UTM Northing (EPSG:32610)
        raster_path: Path to raster file
    
    Returns:
        (col, row) or None if outside bounds.
    """
    with rasterio.open(raster_path) as src:
        # Raster should be in EPSG:32610 (UTM Zone 10N)
        if src.crs != 'EPSG:32610':
            # Transform UTM to raster CRS if needed
            x, y = transform_coords(
                'EPSG:32610',
                src.crs,
                [gcp_x_utm],
                [gcp_y_utm]
            )
            utm_x, utm_y = x[0], y[0]
        else:
            utm_x, utm_y = gcp_x_utm, gcp_y_utm
        
        # Convert to pixel coordinates
        row, col = rasterio.transform.rowcol(src.transform, utm_x, utm_y)
        
        # Check if within bounds
        if 0 <= row < src.height and 0 <= col < src.width:
            return (col, row)
        else:
            return None

# Get basemap CRS and transform
with rasterio.open(basemap_path) as basemap_src:
    basemap_crs = basemap_src.crs
    basemap_transform = basemap_src.transform
    basemap_width = basemap_src.width
    basemap_height = basemap_src.height
    basemap_bounds = basemap_src.bounds

print(f"Basemap CRS: {basemap_crs}")
print(f"Basemap dimensions: {basemap_width}x{basemap_height}")
print(f"Basemap bounds: {basemap_bounds}")
print(f"Basemap transform: {basemap_transform}")

# Convert all GCPs to pixel coordinates
gcp_pixel_coords = {}
for gcp in gcps:
    # Debug: show GCP UTM coordinates
    print(f"\nGCP {gcp['id']}: UTM=({gcp['x_utm']:.2f}, {gcp['y_utm']:.2f})")
    
    pixel_coords = gcp_to_pixel_coords_from_utm(gcp['x_utm'], gcp['y_utm'], basemap_path)
    if pixel_coords:
        gcp_pixel_coords[gcp['id']] = {
            'gcp': gcp,
            'pixel_col': pixel_coords[0],
            'pixel_row': pixel_coords[1],
            'utm_x': gcp.get('x_utm'),
            'utm_y': gcp.get('y_utm'),
        }
        print(f"  ✓ Found at pixel: col={pixel_coords[0]}, row={pixel_coords[1]}")
    else:
        # Debug: show why it's outside bounds
        with rasterio.open(basemap_path) as src:
            row, col = rasterio.transform.rowcol(src.transform, gcp['x_utm'], gcp['y_utm'])
            print(f"  ⚠️  Outside bounds: col={col}, row={row}")
            print(f"     Basemap: {src.width}x{src.height}")
            print(f"     Basemap bounds: {src.bounds}")
            # Check if coordinates are in bounds in UTM space
            in_x = src.bounds.left <= gcp['x_utm'] <= src.bounds.right
            in_y = src.bounds.bottom <= gcp['y_utm'] <= src.bounds.top
            print(f"     UTM X in bounds: {in_x} ({src.bounds.left:.2f} <= {gcp['x_utm']:.2f} <= {src.bounds.right:.2f})")
            print(f"     UTM Y in bounds: {in_y} ({src.bounds.bottom:.2f} <= {gcp['y_utm']:.2f} <= {src.bounds.top:.2f})")

print(f"\n✓ Found {len(gcp_pixel_coords)} GCPs within basemap bounds")
if len(gcp_pixel_coords) > 0:
    print(f"\nFirst few GCP pixel coordinates:")
    for gcp_id, coords in list(gcp_pixel_coords.items())[:3]:
        print(f"  {gcp_id}: col={coords['pixel_col']}, row={coords['pixel_row']}")


Basemap CRS: EPSG:32610
Basemap dimensions: 90129x90188
Basemap bounds: BoundingBox(left=506424.37839793676, bottom=5450017.622213458, right=507501.0951215451, top=5451095.043774429)
Basemap transform: | 0.01, 0.00, 506424.38|
| 0.00,-0.01, 5451095.04|
| 0.00, 0.00, 1.00|

GCP GCP1: UTM=(506914.12, 5450945.53)
  ✓ Found at pixel: col=40995, row=12515

GCP GCP2: UTM=(506657.79, 5450730.01)
  ✓ Found at pixel: col=19538, row=30556

GCP GCP3: UTM=(506577.77, 5450480.01)
  ✓ Found at pixel: col=12840, row=51482

GCP GCP4: UTM=(506765.03, 5450578.63)
  ✓ Found at pixel: col=28515, row=43227

GCP GCP5: UTM=(506926.13, 5450715.96)
  ✓ Found at pixel: col=42000, row=31732

GCP GCP6: UTM=(507071.92, 5450992.66)
  ✓ Found at pixel: col=54203, row=8570

GCP GCP7: UTM=(507089.40, 5450794.23)
  ✓ Found at pixel: col=55667, row=25180

GCP GCP8: UTM=(507315.01, 5450717.85)
  ✓ Found at pixel: col=74551, row=31574

GCP GCP9: UTM=(507252.65, 5450536.03)
  ✓ Found at pixel: col=69332, row=46793

GCP GCP

## Step 4: Extract Patches from Basemap

In [6]:
# Extract patches from basemap centered on GCPs
def extract_patch(raster_path: Path, center_col: int, center_row: int, patch_size: int) -> Optional[np.ndarray]:
    """
    Extract a patch from raster centered on given pixel coordinates.
    
    Args:
        raster_path: Path to raster file
        center_col: Center column (x)
        center_row: Center row (y)
        patch_size: Size of patch (must be odd, e.g., 29, 39, 49)
    
    Returns:
        Patch array (H, W, C) or None if out of bounds
    """
    half_size = patch_size // 2
    
    with rasterio.open(raster_path) as src:
        # Calculate bounds
        col_start = max(0, center_col - half_size)
        col_end = min(src.width, center_col + half_size + 1)
        row_start = max(0, center_row - half_size)
        row_end = min(src.height, center_row + half_size + 1)
        
        # Check if patch would be out of bounds
        if col_end - col_start < patch_size or row_end - row_start < patch_size:
            return None
        
        # Read patch
        patch = src.read(
            window=rasterio.windows.Window(col_start, row_start, col_end - col_start, row_end - row_start)
        )
        
        # Transpose to (H, W, C) format
        if len(patch.shape) == 3:
            patch = np.transpose(patch, (1, 2, 0))
        
        # If single band, convert to 3-channel grayscale
        if len(patch.shape) == 2:
            patch = np.stack([patch, patch, patch], axis=-1)
        
        return patch

def create_gcp_patch_visualization(
    patch: np.ndarray,
    patch_size: int,
    output_path: Path
):
    """
    Create visualization of patch with GCP location marked.
    """
    import matplotlib.pyplot as plt
    import matplotlib.patches as mpatches
    
    # Normalize patch if needed
    if patch.dtype != np.uint8:
        patch_min = patch.min()
        patch_max = patch.max()
        if patch_max > patch_min:
            patch = ((patch - patch_min) / (patch_max - patch_min) * 255).astype(np.uint8)
        else:
            patch = np.zeros_like(patch, dtype=np.uint8)
    
    # Create figure
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    ax.imshow(patch)
    
    # Mark center (GCP location) with bright red dot
    center_row, center_col = patch.shape[0] // 2, patch.shape[1] // 2
    ax.plot(center_col, center_row, 'ro', markersize=15, markeredgewidth=2, markeredgecolor='white')
    
    # Draw yellow square around patch boundary
    rect = mpatches.Rectangle(
        (0, 0), patch.shape[1], patch.shape[0],
        linewidth=3, edgecolor='yellow', facecolor='none'
    )
    ax.add_patch(rect)
    
    ax.set_title(f'Matched Patch ({patch_size}x{patch_size})', fontsize=14, fontweight='bold')
    ax.axis('off')
    
    plt.tight_layout()
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.close()

# Extract patches for different patch sizes
patch_sizes = [49, 59, 79, 99, 119]  # Larger patches for better matching
basemap_patches = {}

for patch_size in patch_sizes:
    basemap_patches[patch_size] = {}
    
    for gcp_id, coords in gcp_pixel_coords.items():
        patch = extract_patch(
            basemap_path,
            coords['pixel_col'],
            coords['pixel_row'],
            patch_size
        )
        
        if patch is not None:
            basemap_patches[patch_size][gcp_id] = patch
            
            # Save patch as image for visualization
            patch_path = patches_dir / f"basemap_{gcp_id}_{patch_size}x{patch_size}.png"
            plt.imsave(patch_path, patch.astype(np.uint8))
    
    print(f"✓ Extracted {len(basemap_patches[patch_size])} patches of size {patch_size}x{patch_size}")

print(f"\n✓ Patch extraction complete!")

✓ Extracted 23 patches of size 49x49
✓ Extracted 23 patches of size 59x59
✓ Extracted 23 patches of size 79x79
✓ Extracted 23 patches of size 99x99
✓ Extracted 23 patches of size 119x119

✓ Patch extraction complete!


## Step 5: Reproject Orthomosaics to Match Basemap CRS

In [7]:
from rasterio.warp import calculate_default_transform, reproject, Resampling, transform_bounds
from rasterio.transform import from_bounds
from rasterio.enums import Resampling as RasterioResampling
from affine import Affine

# Reproject orthos to match basemap CRS and resolution
def reproject_ortho_to_basemap(ortho_path: Path, basemap_path: Path, output_path: Path) -> Path:
    """
    Reproject orthomosaic to match basemap CRS and bounds.
    Uses manual transform construction to avoid CPLE_AppDefinedError.
    """
    if output_path.exists():
        print(f"  ✓ Already reprojected: {output_path}")
        return output_path
    
    with rasterio.open(basemap_path) as basemap_src:
        target_crs = basemap_src.crs
        target_bounds = basemap_src.bounds
        target_transform = basemap_src.transform
        target_width = basemap_src.width
        target_height = basemap_src.height
    
    with rasterio.open(ortho_path) as ortho_src:
        source_crs = ortho_src.crs
        source_bounds = ortho_src.bounds
        
        if source_crs == target_crs:
            print(f"  ✓ Already in target CRS")
            import shutil
            shutil.copy(ortho_path, output_path)
            return output_path
        
        # Transform source bounds to target CRS
        print(f"  Transforming source bounds to target CRS...")
        src_bounds_target_crs = transform_bounds(
            source_crs, target_crs,
            source_bounds.left, source_bounds.bottom,
            source_bounds.right, source_bounds.top
        )
        
        print(f"  Source bounds in target CRS: {src_bounds_target_crs}")
        
        # Get target pixel size
        target_pixel_size_x = abs(target_transform[0])
        target_pixel_size_y = abs(target_transform[4])
        
        # Use intersection of bounds
        output_left = max(src_bounds_target_crs[0], target_bounds.left)
        output_bottom = max(src_bounds_target_crs[1], target_bounds.bottom)
        output_right = min(src_bounds_target_crs[2], target_bounds.right)
        output_top = min(src_bounds_target_crs[3], target_bounds.top)
        
        print(f"  Output bounds (intersection): left={output_left:.2f}, bottom={output_bottom:.2f}, right={output_right:.2f}, top={output_top:.2f}")
        
        # Validate bounds
        if output_right <= output_left or output_top <= output_bottom:
            raise ValueError(f"Invalid output bounds: width={output_right-output_left}, height={output_top-output_bottom}")
        
        # Calculate dimensions using target pixel size
        width = int((output_right - output_left) / target_pixel_size_x)
        height = int((output_top - output_bottom) / target_pixel_size_y)
        
        # Validate dimensions
        if width <= 0 or height <= 0:
            raise ValueError(f"Invalid dimensions: width={width}, height={height}")
        
        # Create transform for output
        transform = Affine.translation(output_left, output_top) * Affine.scale(target_pixel_size_x, -target_pixel_size_y)
        
        print(f"  ✓ Transform calculated: {width}x{height} pixels")
        
        # Read source data
        source_data = ortho_src.read()
        source_count = ortho_src.count
        
        # Reproject
        reprojected_data = np.zeros((source_count, height, width), dtype=source_data.dtype)
        
        for band_idx in range(1, source_count + 1):
            reproject(
                source=rasterio.band(ortho_src, band_idx),
                destination=reprojected_data[band_idx - 1],
                src_transform=ortho_src.transform,
                src_crs=source_crs,
                dst_transform=transform,
                dst_crs=target_crs,
                resampling=Resampling.bilinear
            )
        
        # Save
        with rasterio.open(
            output_path,
            'w',
            driver='GTiff',
            height=height,
            width=width,
            count=source_count,
            dtype=reprojected_data.dtype,
            crs=target_crs,
            transform=transform,
            compress='lzw',
            BIGTIFF='YES',
            tiled=True,
            blockxsize=512,
            blockysize=512
        ) as dst:
            dst.write(reprojected_data)
    
    return output_path

# Check for existing reprojected files from test_matching notebook
existing_reprojected_dir = output_dir / "test_matching" / "reprojected"
reprojected_dir = gcp_matching_dir / "reprojected"
reprojected_dir.mkdir(exist_ok=True)

ortho_paths = {
    'no_gcps': ortho_no_gcps_path,
    'with_gcps': ortho_with_gcps_path
}

reprojected_paths = {}
for ortho_name, ortho_path in ortho_paths.items():
    if not ortho_path.exists():
        print(f"⚠️  Ortho not found: {ortho_path}")
        continue
    
    # Check for existing reprojected file from test_matching
    existing_reprojected = existing_reprojected_dir / f"{ortho_name}_reprojected.tif"
    if existing_reprojected.exists():
        print(f"\nFound existing reprojected file: {existing_reprojected}")
        # Copy to our directory
        import shutil
        reprojected_path = reprojected_dir / f"{ortho_name}_reprojected.tif"
        if not reprojected_path.exists():
            shutil.copy(existing_reprojected, reprojected_path)
            print(f"  ✓ Copied to: {reprojected_path}")
        else:
            print(f"  ✓ Already exists: {reprojected_path}")
        reprojected_paths[ortho_name] = reprojected_path
        continue
    
    # Otherwise, reproject
    print(f"\nReprojecting {ortho_name}...")
    reprojected_path = reproject_ortho_to_basemap(
        ortho_path,
        basemap_path,
        reprojected_dir / f"{ortho_name}_reprojected.tif"
    )
    reprojected_paths[ortho_name] = reprojected_path

print(f"\n✓ Reprojection complete!")


Found existing reprojected file: outputs/test_matching/reprojected/no_gcps_reprojected.tif
  ✓ Already exists: outputs/gcp_matching/reprojected/no_gcps_reprojected.tif

Found existing reprojected file: outputs/test_matching/reprojected/with_gcps_reprojected.tif
  ✓ Already exists: outputs/gcp_matching/reprojected/with_gcps_reprojected.tif

✓ Reprojection complete!


## Step 6: Find GCP Patches in Orthomosaics Using Template Matching

In [8]:
# Find GCP patches in orthomosaics using template matching
def find_patch_in_ortho(
    template_patch: np.ndarray,
    ortho_path: Path,
    search_center_col: int,
    search_center_row: int,
    search_radius: int = 300  # Reduced for more precise matching
) -> Optional[Tuple[int, int, float]]:
    """
    Find template patch in orthomosaic using template matching.
    
    Returns:
        (col, row, confidence) or None if not found
    """
    # Convert template to grayscale if needed
    if len(template_patch.shape) == 3:
        template_gray = cv2.cvtColor(template_patch.astype(np.uint8), cv2.COLOR_RGB2GRAY)
    else:
        template_gray = template_patch.astype(np.uint8)
    
    with rasterio.open(ortho_path) as ortho_src:
        # Define search window
        search_col_start = max(0, search_center_col - search_radius)
        search_col_end = min(ortho_src.width, search_center_col + search_radius)
        search_row_start = max(0, search_center_row - search_radius)
        search_row_end = min(ortho_src.height, search_center_row + search_radius)
        
        # Read search region
        search_window = rasterio.windows.Window(
            search_col_start,
            search_row_start,
            search_col_end - search_col_start,
            search_row_end - search_row_start
        )
        
        search_region = ortho_src.read(window=search_window)
        
        # Convert to (H, W, C) and then grayscale
        if len(search_region.shape) == 3:
            search_region = np.transpose(search_region, (1, 2, 0))
            if search_region.shape[2] == 1:
                search_gray = search_region[:, :, 0]
            else:
                search_gray = cv2.cvtColor(search_region.astype(np.uint8), cv2.COLOR_RGB2GRAY)
        else:
            search_gray = search_region
        
        # Normalize to uint8
        if search_gray.dtype != np.uint8:
            search_min = search_gray.min()
            search_max = search_gray.max()
            if search_max > search_min:
                search_gray = ((search_gray - search_min) / (search_max - search_min) * 255).astype(np.uint8)
            else:
                search_gray = np.zeros_like(search_gray, dtype=np.uint8)
        
        # Template matching
        result = cv2.matchTemplate(search_gray, template_gray, cv2.TM_CCOEFF_NORMED)
        
        # Find best match
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
        
        # Convert back to global coordinates
        match_col = search_col_start + max_loc[0] + template_gray.shape[1] // 2
        match_row = search_row_start + max_loc[1] + template_gray.shape[0] // 2
        
        # Return if confidence is high enough
        if max_val > 0.5:  # Threshold for match confidence
            return (match_col, match_row, float(max_val))
        else:
            return None

# Find GCPs in each orthomosaic
# Create directory for matching patches
matching_patches_dir = gcp_matching_dir / "matching_patches"
matching_patches_dir.mkdir(exist_ok=True)
matching_patches_dir.mkdir(exist_ok=True)

matching_results = {}

for ortho_name, reprojected_path in reprojected_paths.items():

    matching_results[ortho_name] = {}
    
    # Get ortho transform for coordinate conversion
    with rasterio.open(reprojected_path) as ortho_src:
        ortho_transform = ortho_src.transform
    
    # Try different patch sizes
    best_patch_size = None
    best_matches = 0
    
    for patch_size in patch_sizes:
        matches_found = 0
        
        for gcp_id, coords in gcp_pixel_coords.items():
            if gcp_id not in basemap_patches[patch_size]:
                continue
            
            template = basemap_patches[patch_size][gcp_id]
            
            # Convert GCP UTM coordinates to pixel coordinates in THIS ortho
            gcp_utm_x = coords.get('utm_x') or coords.get('x_utm')
            gcp_utm_y = coords.get('utm_y') or coords.get('y_utm')
            
            if gcp_utm_x is not None and gcp_utm_y is not None:
                # Convert UTM to pixel coordinates using ortho's transform
                expected_col, expected_row = ~ortho_transform * (gcp_utm_x, gcp_utm_y)
                expected_col = int(expected_col)
                expected_row = int(expected_row)
            else:
                # Fallback: use pixel coordinates from basemap
                expected_col = coords['pixel_col']
                expected_row = coords['pixel_row']
            
            # Search for patch using multi-scale matching
            if 'find_patch_in_ortho_multiscale' in globals():
                match = find_patch_in_ortho_multiscale(
                    template,
                    reprojected_path,
                    expected_col,
                    expected_row,
                    search_radius=300
                )
            else:
                match = find_patch_in_ortho(
                    template,
                    reprojected_path,
                    expected_col,
                    expected_row,
                    search_radius=300
                )
            
            # Validate match quality
            if match and match[2] < 0.3:  # Confidence threshold
                match = None
            
            if match:
                match_col, match_row, confidence = match
                matches_found += 1
                
                if gcp_id not in matching_results[ortho_name]:
                    matching_results[ortho_name][gcp_id] = {}
                
                matching_results[ortho_name][gcp_id][patch_size] = {
                    'expected_col': expected_col,
                    'expected_row': expected_row,
                    'matched_col': match_col,
                    'matched_row': match_row,
                    'offset_col': match_col - expected_col,
                    'offset_row': match_row - expected_row,
                    'confidence': confidence
                }
        
        print(f"  Patch size {patch_size}x{patch_size}: {matches_found}/{len(gcp_pixel_coords)} matches")
        
        if matches_found > best_matches:
            best_matches = matches_found
            best_patch_size = patch_size
    
    print(f"\n  ✓ Best patch size: {best_patch_size}x{best_patch_size} ({best_matches} matches)")

    print(f"\n  ✓ Best patch size: {best_patch_size}x{best_patch_size} ({best_matches} matches)")

    # Create subdirectory for this ortho's matching patches
    ortho_patches_dir = matching_patches_dir / ortho_name
    ortho_patches_dir.mkdir(exist_ok=True)

    # Save matching patches for visual verification
    print(f"  Saving matching patches to {ortho_patches_dir}...")
    for gcp_id, match_data in matching_results[ortho_name].items():
        if best_patch_size in match_data:
            match = match_data[best_patch_size]
            
            # Extract patch from ortho at matched location
            matched_col = match['matched_col']
            matched_row = match['matched_row']
            
            # Extract patch (same size as template)
            patch = extract_patch(
                reprojected_path,
                matched_col,
                matched_row,
                best_patch_size
            )
            
            if patch is not None:
                # Normalize patch for saving
                if patch.dtype != np.uint8:
                    patch_min = patch.min()
                    patch_max = patch.max()
                    if patch_max > patch_min:
                        patch = ((patch - patch_min) / (patch_max - patch_min) * 255).astype(np.uint8)
                    else:
                        patch = np.zeros_like(patch, dtype=np.uint8)
                
                # Save matching patch
                match_patch_path = ortho_patches_dir / f"{gcp_id}_{best_patch_size}x{best_patch_size}_matched.png"
                plt.imsave(match_patch_path, patch)
                
                # Also create visualization with GCP location marked
                vis_patch_path = ortho_patches_dir / f"{gcp_id}_{best_patch_size}x{best_patch_size}_matched_vis.png"
                create_gcp_patch_visualization(patch, best_patch_size, vis_patch_path)
    
    print(f"  ✓ Saved {len([g for g in matching_results[ortho_name].keys() if best_patch_size in matching_results[ortho_name][g]])} matching patches")

print(f"\n✓ Patch matching complete!")

# Create comprehensive visualization
print(f"\nCreating visualization of GCP matches...")

def create_gcp_matching_visualization(
    basemap_path: Path,
    ortho_paths: Dict[str, Path],
    gcp_pixel_coords: Dict,
    matching_results: Dict,
    output_path: Path,
    max_dimension: int = 4000
):
    """
    Create visualization showing basemap with GCPs and orthos with matched patches.
    """
    # Load basemap
    with rasterio.open(basemap_path) as src:
        basemap_data = src.read()
        basemap_transform = src.transform
        
        # Convert to (H, W, C)
        if len(basemap_data.shape) == 3:
            basemap_img = np.transpose(basemap_data, (1, 2, 0))
            if basemap_img.shape[2] == 1:
                basemap_img = np.stack([basemap_img[:, :, 0]] * 3, axis=-1)
            elif basemap_img.shape[2] == 4:
                basemap_img = basemap_img[:, :, :3]  # Take RGB
        else:
            basemap_img = np.stack([basemap_data] * 3, axis=-1)
        
        # Normalize to uint8
        if basemap_img.dtype != np.uint8:
            basemap_min = basemap_img.min()
            basemap_max = basemap_img.max()
            if basemap_max > basemap_min:
                basemap_img = ((basemap_img - basemap_min) / (basemap_max - basemap_min) * 255).astype(np.uint8)
            else:
                basemap_img = np.zeros_like(basemap_img, dtype=np.uint8)
    
    # Downsample if too large
    h, w = basemap_img.shape[:2]
    if max(h, w) > max_dimension:
        scale = max_dimension / max(h, w)
        new_h, new_w = int(h * scale), int(w * scale)
        basemap_img = cv2.resize(basemap_img, (new_w, new_h), interpolation=cv2.INTER_AREA)
        scale_factor = scale
    else:
        scale_factor = 1.0
    
    # Load orthos and create panels
    num_orthos = len(ortho_paths)
    fig, axes = plt.subplots(1, num_orthos + 1, figsize=(8 * (num_orthos + 1), 8))
    
    # Basemap panel (left)
    ax = axes[0]
    basemap_display = basemap_img.copy()
    
    # Draw GCP positions on basemap
    for gcp_id, coords in gcp_pixel_coords.items():
        # Scale coordinates
        col = int(coords['pixel_col'] * scale_factor)
        row = int(coords['pixel_row'] * scale_factor)
        
        if 0 <= row < basemap_display.shape[0] and 0 <= col < basemap_display.shape[1]:
            # Draw red circle
            cv2.circle(basemap_display, (col, row), 10, (255, 0, 0), 3)
    
    ax.imshow(basemap_display)
    ax.set_title('Basemap with GCP Locations', fontsize=14, fontweight='bold')
    
    # Add GCP labels
    for gcp_id, coords in gcp_pixel_coords.items():
        col = int(coords['pixel_col'] * scale_factor)
        row = int(coords['pixel_row'] * scale_factor)
        if 0 <= row < basemap_display.shape[0] and 0 <= col < basemap_display.shape[1]:
            ax.text(col, row - 15, gcp_id, color='red', fontsize=8, fontweight='bold',
                   ha='center', va='bottom')
    
    ax.axis('off')
    
    # Ortho panels (right)
    for ortho_idx, (ortho_name, ortho_path) in enumerate(ortho_paths.items(), 1):
        ax = axes[ortho_idx]
        
        # Load ortho
        with rasterio.open(ortho_path) as src:
            ortho_data = src.read()
            
            # Convert to (H, W, C)
            if len(ortho_data.shape) == 3:
                ortho_img = np.transpose(ortho_data, (1, 2, 0))
                if ortho_img.shape[2] == 1:
                    ortho_img = np.stack([ortho_img[:, :, 0]] * 3, axis=-1)
                elif ortho_img.shape[2] == 4:
                    ortho_img = ortho_img[:, :, :3]
            else:
                ortho_img = np.stack([ortho_data] * 3, axis=-1)
            
            # Normalize
            if ortho_img.dtype != np.uint8:
                ortho_min = ortho_img.min()
                ortho_max = ortho_img.max()
                if ortho_max > ortho_min:
                    ortho_img = ((ortho_img - ortho_min) / (ortho_max - ortho_min) * 255).astype(np.uint8)
                else:
                    ortho_img = np.zeros_like(ortho_img, dtype=np.uint8)
        
        # Downsample if too large
        h, w = ortho_img.shape[:2]
        if max(h, w) > max_dimension:
            scale = max_dimension / max(h, w)
            new_h, new_w = int(h * scale), int(w * scale)
            ortho_img = cv2.resize(ortho_img, (new_w, new_h), interpolation=cv2.INTER_AREA)
            ortho_scale = scale
        else:
            ortho_scale = 1.0
        
        ortho_display = ortho_img.copy()
        
        # Draw matched patch centers
        if ortho_name in matching_results:
            for gcp_id, match_data in matching_results[ortho_name].items():
                # Get best patch size match
                best_patch_size = max(match_data.keys()) if match_data else None
                if best_patch_size:
                    match = match_data[best_patch_size]
                    matched_col = int(match['matched_col'] * ortho_scale)
                    matched_row = int(match['matched_row'] * ortho_scale)
                    
                    if 0 <= matched_row < ortho_display.shape[0] and 0 <= matched_col < ortho_display.shape[1]:
                        # Draw yellow circle
                        cv2.circle(ortho_display, (matched_col, matched_row), 10, (255, 255, 0), 3)
        
        ax.imshow(ortho_display)
        ax.set_title(f'{ortho_name.replace("_", " ").title()} with Matched Patches', fontsize=14, fontweight='bold')
        
        # Add labels
        if ortho_name in matching_results:
            for gcp_id, match_data in matching_results[ortho_name].items():
                best_patch_size = max(match_data.keys()) if match_data else None
                if best_patch_size:
                    match = match_data[best_patch_size]
                    matched_col = int(match['matched_col'] * ortho_scale)
                    matched_row = int(match['matched_row'] * ortho_scale)
                    if 0 <= matched_row < ortho_display.shape[0] and 0 <= matched_col < ortho_display.shape[1]:
                        ax.text(matched_col, matched_row - 15, gcp_id, color='yellow', fontsize=8, fontweight='bold',
                               ha='center', va='bottom')
        
        ax.axis('off')
    
    plt.tight_layout()
    plt.savefig(output_path, dpi=300, bbox_inches='tight', format='PNG')
    plt.close()
    
    print(f"✓ Visualization saved: {output_path}")

# Create visualization for each ortho
for ortho_name in reprojected_paths.keys():
    if ortho_name not in matching_results:
        continue
    
    vis_path = matches_dir / f"gcp_matching_visualization_{ortho_name}.png"

    # Check if visualization already exists
    if vis_path.exists():
        print(f"  ✓ Visualization already exists: {vis_path}")
        print(f"  Skipping visualization creation...")
        continue

    
    create_gcp_matching_visualization(
        basemap_path,
        {ortho_name: reprojected_paths[ortho_name]},
        gcp_pixel_coords,
        matching_results,
        vis_path,
        max_dimension=4000
    )


  Patch size 49x49: 21/23 matches
  Patch size 59x59: 21/23 matches
  Patch size 79x79: 19/23 matches
  Patch size 99x99: 18/23 matches
  Patch size 119x119: 18/23 matches

  ✓ Best patch size: 49x49 (21 matches)

  ✓ Best patch size: 49x49 (21 matches)
  Saving matching patches to outputs/gcp_matching/matching_patches/no_gcps...
  ✓ Saved 21 matching patches
  Patch size 49x49: 21/23 matches
  Patch size 59x59: 20/23 matches
  Patch size 79x79: 19/23 matches
  Patch size 99x99: 18/23 matches
  Patch size 119x119: 17/23 matches

  ✓ Best patch size: 49x49 (21 matches)

  ✓ Best patch size: 49x49 (21 matches)
  Saving matching patches to outputs/gcp_matching/matching_patches/with_gcps...
  ✓ Saved 21 matching patches

✓ Patch matching complete!

Creating visualization of GCP matches...
  ✓ Visualization already exists: outputs/gcp_matching/matches/gcp_matching_visualization_no_gcps.png
  Skipping visualization creation...
  ✓ Visualization already exists: outputs/gcp_matching/matches/gc

## Step 7: Compute 2D Shift or Affine Transformation

In [12]:
import numpy as np
from typing import Dict, List, Optional, Tuple
import cv2

def remove_outliers_ransac(src_points: np.ndarray, dst_points: np.ndarray, threshold: float = 50.0, min_samples: int = 3) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Remove outliers using RANSAC with proper model fitting.
    
    Returns:
        (inlier_src, inlier_dst, inlier_mask)
    """
    
    if len(src_points) < min_samples:
        mask = np.ones(len(src_points), dtype=bool)
        return src_points, dst_points, mask
    
    # Convert to numpy arrays if needed
    src_points = np.array(src_points, dtype=np.float32)
    dst_points = np.array(dst_points, dtype=np.float32)
    
    # Use RANSAC for X and Y separately, then combine
    # For 2D shift, we fit a simple translation model
    # Compute median shift as initial estimate
    shifts = dst_points - src_points
    median_shift = np.median(shifts, axis=0)
    
    # Compute distances from median shift
    expected_dst = src_points + median_shift
    distances = np.sqrt(np.sum((dst_points - expected_dst)**2, axis=1))
    
    # Use IQR method for outlier detection
    q1 = np.percentile(distances, 25)
    q3 = np.percentile(distances, 75)
    iqr = q3 - q1
    outlier_threshold = q3 + 2.5 * iqr  # More aggressive (was 1.5)
    
    # Also use absolute threshold (in pixels)
    absolute_threshold = max(threshold, 100.0)  # At least 100 pixels
    
    # Mark outliers
    inlier_mask = (distances <= outlier_threshold) & (distances <= absolute_threshold)
    
    # Ensure we have at least min_samples inliers
    if np.sum(inlier_mask) < min_samples:
        # Keep the min_samples points closest to the median
        sorted_indices = np.argsort(distances)
        inlier_mask = np.zeros(len(src_points), dtype=bool)
        inlier_mask[sorted_indices[:min_samples]] = True
    
    # Ensure inlier_mask is a proper boolean array
    inlier_mask = np.asarray(inlier_mask, dtype=bool)
    
    # Return filtered points
    return src_points[inlier_mask], dst_points[inlier_mask], inlier_mask

def compute_transformation(matches: Dict, transformation_type: str = 'shift', match_distances: Optional[List[float]] = None) -> Dict:
    """
    Compute transformation from GCP matches.
    
    Args:
        matches: Dictionary with GCP matches
        transformation_type: 'shift', 'affine', 'homography', or 'deformable'
        match_distances: Optional list of match distances for RANSAC weighting
    
    Returns:
        Dictionary with transformation parameters
    """
    # Collect source and destination points
    src_points = []
    dst_points = []
    
    for gcp_id, match_data in matches.items():
        # Use the best patch size match
        best_patch_size = max(match_data.keys())
        match = match_data[best_patch_size]
        
        src_points.append([match['expected_col'], match['expected_row']])
        dst_points.append([match['matched_col'], match['matched_row']])
    
    src_points = np.array(src_points, dtype=np.float32)
    dst_points = np.array(dst_points, dtype=np.float32)

    # Remove outliers using RANSAC with distance weighting
    src_points, dst_points, inlier_mask = remove_outliers_ransac(src_points, dst_points, threshold=100.0, min_samples=3)
    
    if len(src_points) < 3:
        return {'type': 'insufficient_points', 'error': 'Need at least 3 matches after outlier removal'}
    
    # Compute transformation based on type
    if transformation_type == 'shift':
        # Compute 2D shift (mean offset)
        offsets = dst_points - src_points
        shift_x = float(np.mean(offsets[:, 0]))
        shift_y = float(np.mean(offsets[:, 1]))
        
        # Compute RMSE
        errors = offsets - np.array([shift_x, shift_y])
        rmse = float(np.sqrt(np.mean(np.sum(errors**2, axis=1))))
        
        return {
            'type': 'shift',
            'shift_x': shift_x,
            'shift_y': shift_y,
            'rmse': rmse,
            'num_points': len(src_points)
        }
    
    elif transformation_type == 'affine':
        # Compute affine transformation using least squares (all points)
        if len(src_points) < 3:
            return {'type': 'insufficient_points', 'error': 'Need at least 3 points for affine'}
        
        # Build system: A * params = b
        A = np.zeros((2 * len(src_points), 6))
        b = np.zeros(2 * len(src_points))
        
        for k in range(len(src_points)):
            x, y = src_points[k]
            xp, yp = dst_points[k]
            A[2*k, :] = [x, y, 1, 0, 0, 0]
            b[2*k] = xp
            A[2*k+1, :] = [0, 0, 0, x, y, 1]
            b[2*k+1] = yp
        
        # Solve using least squares
        params, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None)
        transform_matrix = params.reshape(2, 3)
        
        # Apply to all points to compute error
        ones = np.ones((len(src_points), 1))
        src_homogeneous = np.hstack([src_points, ones])
        transformed = (transform_matrix @ src_homogeneous.T).T
        
        errors = dst_points - transformed
        rmse = float(np.sqrt(np.mean(np.sum(errors**2, axis=1))))
        
        return {
            'type': 'affine',
            'matrix': transform_matrix.tolist(),
            'rmse': rmse,
            'num_points': len(src_points)
        }
    
    elif transformation_type == 'homography':
        # Compute homography transformation (8 parameters, requires at least 4 points)
        if len(src_points) < 4:
            return {'type': 'insufficient_points', 'error': 'Need at least 4 points for homography'}
        
        # Use cv2.findHomography with RANSAC
        try:
            homography_matrix, inlier_mask = cv2.findHomography(
                src_points.reshape(-1, 1, 2),
                dst_points.reshape(-1, 1, 2),
                method=cv2.RANSAC,
                ransacReprojThreshold=5.0,
                maxIters=2000,
                confidence=0.99
            )
            
            if homography_matrix is not None:
                inlier_src = src_points[inlier_mask.ravel() == 1]
                
                # Apply to all points to compute error
                ones = np.ones((len(src_points), 1))
                src_homogeneous = np.hstack([src_points, ones])
                transformed = (homography_matrix @ src_homogeneous.T).T
                transformed = transformed[:, :2] / transformed[:, 2:3]
                
                errors = dst_points - transformed
                rmse = float(np.sqrt(np.mean(np.sum(errors**2, axis=1))))
                
                return {
                    'type': 'homography',
                    'matrix': homography_matrix.tolist(),
                    'rmse': rmse,
                    'num_points': len(inlier_src),
                    'num_inliers': int(np.sum(inlier_mask))
                }
            else:
                return {'type': 'homography_failed', 'error': 'Homography computation failed'}
        except Exception as e:
            return {'type': 'homography_error', 'error': str(e)}
    
    elif transformation_type == 'deformable':
        # Compute deformable (curvilinear) transformation using thin-plate spline
        if len(src_points) < 3:
            return {'type': 'insufficient_points', 'error': 'Need at least 3 points for deformable transformation'}
        
        try:
            from scipy.interpolate import RBFInterpolator
            
            # Use RANSAC to select inliers based on distance
            if match_distances is not None and len(match_distances) == len(src_points):
                # Weight by inverse distance (lower distance = higher weight)
                weights = 1.0 / (np.array(match_distances) + 1e-6)
                weights = weights / weights.sum()
                # Select top 80% of points by weight (inliers)
                sorted_indices = np.argsort(weights)[::-1]
                num_inliers = max(3, int(0.8 * len(src_points)))
                inlier_indices = sorted_indices[:num_inliers]
                inlier_src = src_points[inlier_indices]
                inlier_dst = dst_points[inlier_indices]
            else:
                inlier_src = src_points
                inlier_dst = dst_points
            
            # Fit RBF interpolator (thin-plate spline)
            rbf = RBFInterpolator(inlier_src, inlier_dst, kernel='thin_plate_spline', smoothing=0.0)
            
            # Evaluate on all points to compute error
            transformed = rbf(src_points)
            errors = dst_points - transformed
            rmse = float(np.sqrt(np.mean(np.sum(errors**2, axis=1))))
            
            return {
                'type': 'deformable',
                'rmse': rmse,
                'num_points': len(inlier_src),
                'inlier_indices': inlier_indices.tolist() if 'inlier_indices' in locals() else None,
                'src_points': inlier_src.tolist() if 'inlier_src' in locals() else src_points.tolist(),
                'dst_points': inlier_dst.tolist() if 'inlier_dst' in locals() else dst_points.tolist()
            }
        except ImportError:
            return {'type': 'deformable_error', 'error': 'scipy.interpolate.RBFInterpolator not available'}
        except Exception as e:
            return {'type': 'deformable_error', 'error': str(e)}
    
    else:
        return {'type': 'unknown', 'error': f'Unknown transformation type: {transformation_type}'}

# Compute transformations for each ortho
print("=" * 60)
print("Step 7: Compute Transformations")
print("=" * 60)

# Check if matching_results is defined, load from file if not
try:
    _ = matching_results
    print("✓ matching_results found in memory")
except NameError:
    print("matching_results not in memory, attempting to load from file...")
    from pathlib import Path
    import json
    try:
        try:
            _ = output_dir
        except NameError:
            output_dir = Path("outputs")
        try:
            _ = gcp_matching_dir
        except NameError:
            gcp_matching_dir = output_dir / "gcp_matching"
        matches_dir = gcp_matching_dir / "matches"
        
        matching_results_file = matches_dir / "matching_results.json"
        if matching_results_file.exists():
            with open(matching_results_file, 'r') as f:
                matching_results = json.load(f)
            print(f"✓ Loaded matching_results from {matching_results_file}")
        else:
            raise FileNotFoundError(f"matching_results.json not found at {matching_results_file}")
    except Exception as e:
        print(f"❌ Could not load matching_results: {e}")
        print("Please run Step 6 (Patch Matching) first.")
        raise

# Compute transformations for each ortho
transformations = {}

for ortho_name in matching_results.keys():
    print(f"\nProcessing {ortho_name}...")
    
    # Get matches for this ortho
    matches = matching_results[ortho_name]
    
    if len(matches) < 3:
        print(f"  ⚠️  Insufficient matches ({len(matches)}) for {ortho_name}")
        transformations[ortho_name] = {'error': f'Insufficient matches: {len(matches)}'}
        continue
    
    # Extract match distances for RANSAC weighting
    match_distances = []
    for gcp_id, match_data in matches.items():
        best_patch_size = max(match_data.keys())
        match = match_data[best_patch_size]
        if 'distance_m' in match:
            match_distances.append(match['distance_m'])
    
    if len(match_distances) != len(matches):
        match_distances = None
    
    # Compute all transformation types
    transformation_results = {}
    
    for trans_type in ['shift', 'affine', 'homography', 'deformable']:
        try:
            result = compute_transformation(matches, trans_type, match_distances)
            if 'error' not in result:
                transformation_results[trans_type] = result
                print(f"  ✓ {trans_type}: RMSE = {result.get('rmse', 'N/A'):.2f} pixels")
            else:
                print(f"  ⚠️  {trans_type}: {result.get('error', 'Unknown error')}")
        except Exception as e:
            print(f"  ❌ {trans_type} failed: {e}")
    
    # Select top two transformations by RMSE
    if len(transformation_results) == 0:
        print(f"  ❌ No valid transformations for {ortho_name}")
        transformations[ortho_name] = {'error': 'No valid transformations'}
        continue
    
    # Sort by RMSE (lower is better)
    sorted_transforms = sorted(
        transformation_results.items(),
        key=lambda x: x[1].get('rmse', float('inf'))
    )
    
    # Primary (best) transformation
    primary_type, primary_trans = sorted_transforms[0]
    print(f"\n  Primary: {primary_type} (RMSE: {primary_trans.get('rmse', 'N/A'):.2f} pixels)")
    
    # Secondary (second best) transformation if available
    secondary_trans = None
    if len(sorted_transforms) > 1:
        secondary_type, secondary_trans = sorted_transforms[1]
        print(f"  Secondary: {secondary_type} (RMSE: {secondary_trans.get('rmse', 'N/A'):.2f} pixels)")
    
    # Store transformations
    transformations[ortho_name] = {
        'primary': primary_trans,
        'secondary': secondary_trans
    }

# Save transformations to file
try:
    _ = matches_dir
except NameError:
    from pathlib import Path
    try:
        _ = output_dir
    except NameError:
        output_dir = Path("outputs")
    try:
        _ = gcp_matching_dir
    except NameError:
        gcp_matching_dir = output_dir / "gcp_matching"
    matches_dir = gcp_matching_dir / "matches"
    matches_dir.mkdir(parents=True, exist_ok=True)

transformations_file = matches_dir / "transformations.json"

# Convert numpy types to native Python types for JSON serialization
def convert_to_native_types(obj):
    """Recursively convert numpy types to native Python types."""
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {key: convert_to_native_types(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_native_types(item) for item in obj]
    return obj

transformations_serializable = convert_to_native_types(transformations)

with open(transformations_file, 'w') as f:
    json.dump(transformations_serializable, f, indent=2)

print(f"\n✓ Transformations saved to {transformations_file}")
print(f"  Total orthos processed: {len(transformations)}")


## Step 8: Apply Transformation and Register Orthomosaics

In [10]:
# Apply transformation to orthomosaic
def apply_transformation(
    ortho_path: Path,
    transformation: Dict,
    output_path: Path,
    basemap_path: Path
) -> Path:
    """
    Apply transformation to register orthomosaic to basemap.
    """
    with rasterio.open(basemap_path) as basemap_src:
        target_width = basemap_src.width
        target_height = basemap_src.height
        target_transform = basemap_src.transform
        target_crs = basemap_src.crs
    
    with rasterio.open(ortho_path) as ortho_src:
        source_data = ortho_src.read()
        source_count = ortho_src.count
        
        # Apply transformation
        if transformation['type'] == 'shift':
            # Apply 2D shift using scipy
            shift_x = transformation['shift_x']
            shift_y = transformation['shift_y']
            
            registered_data = np.zeros((source_count, target_height, target_width), dtype=source_data.dtype)
            
            for band_idx in range(source_count):
                shifted = ndimage.shift(
                    source_data[band_idx],
                    (shift_y, shift_x),
                    mode='constant',
                    cval=0,
                    order=1
                )
                
                # Crop or pad to match target dimensions
                if shifted.shape[0] > target_height:
                    shifted = shifted[:target_height, :]
                elif shifted.shape[0] < target_height:
                    padded = np.zeros((target_height, shifted.shape[1]), dtype=shifted.dtype)
                    padded[:shifted.shape[0], :] = shifted
                    shifted = padded
                
                if shifted.shape[1] > target_width:
                    shifted = shifted[:, :target_width]
                elif shifted.shape[1] < target_width:
                    padded = np.zeros((target_height, target_width), dtype=shifted.dtype)
                    padded[:, :shifted.shape[1]] = shifted
                    shifted = padded
                
                registered_data[band_idx] = shifted
        
        elif transformation['type'] == 'affine':
            # Apply affine transformation using scipy (handles large images)
            transform_matrix = np.array(transformation['matrix'], dtype=np.float32)
            
            # Extract transformation components
            matrix_2x2 = transform_matrix[:2, :2]
            offset = transform_matrix[:2, 2]
            
            # Create output array
            registered_data = np.zeros((source_count, target_height, target_width), dtype=source_data.dtype)
            
            # Apply transformation per band
            for band_idx in range(source_count):
                transformed = ndimage.affine_transform(
                    source_data[band_idx],
                    matrix=matrix_2x2.T,
                    offset=offset[::-1],
                    output_shape=(target_height, target_width),
                    order=1,
                    mode='constant',
                    cval=0
                )
                registered_data[band_idx] = transformed

                registered_data[band_idx] = transformed




        elif transformation['type'] == 'homography':
            # Apply homography transformation
            homography_matrix = np.array(transformation['matrix'], dtype=np.float32)
            
            # Create output array
            registered_data = np.zeros((source_count, target_height, target_width), dtype=source_data.dtype)
            
            # Apply transformation per band
            for band_idx in range(source_count):
                # Use cv2.warpPerspective for smaller images, scipy for large ones
                if target_height < 32767 and target_width < 32767:
                    transformed = cv2.warpPerspective(
                        source_data[band_idx].astype(np.float32),
                        homography_matrix,
                        (target_width, target_height),
                        flags=cv2.INTER_LINEAR,
                        borderMode=cv2.BORDER_CONSTANT,
                        borderValue=0
                    )
                    registered_data[band_idx] = transformed.astype(source_data.dtype)

                else:
                    # Use scipy for large images
                    y_coords, x_coords = np.mgrid[0:target_height, 0:target_width].astype(np.float32)
                    coords = np.stack([x_coords.ravel(), y_coords.ravel(), np.ones(target_height * target_width)]).T
                    inv_homography = np.linalg.inv(homography_matrix)
                    src_coords = (inv_homography @ coords.T).T
                    src_coords = src_coords[:, :2] / src_coords[:, 2:3]
                    src_x = src_coords[:, 0].reshape(target_height, target_width)
                    src_y = src_coords[:, 1].reshape(target_height, target_width)
                    transformed = ndimage.map_coordinates(
                        source_data[band_idx],
                        [src_y, src_x],
                        order=1,
                        mode='constant',
                        cval=0
                    )
                    registered_data[band_idx] = transformed.astype(source_data.dtype)

        elif transformation['type'] == 'deformable':
            # Apply deformable (thin-plate spline) transformation
            from scipy.interpolate import RBFInterpolator
        
            # Recreate RBF interpolator from stored points
            src_points = np.array(transformation.get('src_points', []), dtype=np.float32)
            dst_points = np.array(transformation.get('dst_points', []), dtype=np.float32)
        
            if len(src_points) < 3:
                raise ValueError('Insufficient points for deformable transformation')
        
            # Fit RBF interpolator
            rbf = RBFInterpolator(src_points, dst_points, kernel='thin_plate_spline', smoothing=0.0)
        
            # Create output array
            registered_data = np.zeros((source_count, target_height, target_width), dtype=source_data.dtype)
        
            # Create coordinate grids for target image
            y_coords, x_coords = np.mgrid[0:target_height, 0:target_width].astype(np.float32)
        
            # Convert pixel coordinates to source image coordinates using inverse transform
            # We need to map target pixels back to source pixels
            # The RBF maps from source to destination, so we need to invert it
            # For simplicity, we'll use the forward mapping: source -> target
        
            # Apply transformation per band
            for band_idx in range(source_count):
                # For each target pixel, find corresponding source pixel
                # We need to invert the RBF: find source coords that map to target coords
                # This is complex, so we'll use a simpler approach:
                # Map source image through RBF to get warped coordinates, then resample
            
                # Create source coordinate grid
            src_y, src_x = np.mgrid[0:source_height, 0:source_width].astype(np.float32)
            
            # Apply RBF to get warped coordinates
            src_coords = np.stack([src_x.ravel(), src_y.ravel()], axis=1)
            warped_coords = rbf(src_coords)
            
            # Reshape warped coordinates
            warped_x = warped_coords[:, 0].reshape(source_height, source_width)
            warped_y = warped_coords[:, 1].reshape(source_height, source_width)
            
            # Map warped source to target grid using scipy.ndimage.map_coordinates
            # We need to find where each target pixel maps to in the warped source
            from scipy import ndimage
            
            # For deformable transformation, we'll use a simpler approach:
            # Interpolate the warped source image onto the target grid
            # Create a temporary warped image
            warped_band = ndimage.map_coordinates(
                source_data[band_idx],
                [warped_y, warped_x],
                order=1,
                mode='constant',
                cval=0
            )
            
            # Now resample warped_band to target dimensions
            if warped_band.shape != (target_height, target_width):
                from scipy.ndimage import zoom
                zoom_y = target_height / warped_band.shape[0]
                zoom_x = target_width / warped_band.shape[1]
                warped_band = zoom(warped_band, (zoom_y, zoom_x), order=1)
            
            registered_data[band_idx] = warped_band[:target_height, :target_width].astype(source_data.dtype)


            # Apply deformable (thin-plate spline) transformation
            from scipy.interpolate import RBFInterpolator
            from scipy import ndimage
        
            # Recreate RBF interpolator from stored points
            src_points = np.array(transformation.get('src_points', []), dtype=np.float32)
            dst_points = np.array(transformation.get('dst_points', []), dtype=np.float32)
        
            if len(src_points) < 3:
            raise ValueError('Insufficient points for deformable transformation')
        
            # Fit RBF interpolator
            rbf = RBFInterpolator(src_points, dst_points, kernel='thin_plate_spline', smoothing=0.0)
        
            # Create output array
            registered_data = np.zeros((source_count, target_height, target_width), dtype=source_data.dtype)
        
            # Apply transformation per band
            for band_idx in range(source_count):
            # Create source coordinate grid
            src_y, src_x = np.mgrid[0:source_height, 0:source_width].astype(np.float32)
            
            # Apply RBF to get warped coordinates
            src_coords = np.stack([src_x.ravel(), src_y.ravel()], axis=1)
            warped_coords = rbf(src_coords)
            
            # Reshape warped coordinates
            warped_x = warped_coords[:, 0].reshape(source_height, source_width)
            warped_y = warped_coords[:, 1].reshape(source_height, source_width)
            
            # Map coordinates to sample from source
            # The warped coordinates tell us where each source pixel maps to
            # We need to invert this to find which source pixels map to each target pixel
            # For simplicity, we'll create a warped image and then resample it
            
            # Create target coordinate grid
            tgt_y, tgt_x = np.mgrid[0:target_height, 0:target_width].astype(np.float32)
            
            # For each target pixel, find corresponding source pixel
            # We need to solve: rbf(src_coord) = tgt_coord for src_coord
            # This is complex, so we'll use a simpler approach:
            # 1. Create a mapping from source to warped coordinates
            # 2. For each target pixel, find nearest warped coordinate and use corresponding source
            
            # Simplified: use the warped coordinates directly to resample
            # Map warped coordinates back to source space (inverse mapping)
            # For now, use a simpler approach: resample the source using warped coordinates
            
            # Create output by sampling source at warped coordinates
            # Clamp warped coordinates to source bounds
            warped_x_clamped = np.clip(warped_x, 0, source_width - 1)
            warped_y_clamped = np.clip(warped_y, 0, source_height - 1)
            
            # Sample from source using warped coordinates
            warped_band = ndimage.map_coordinates(
                source_data[band_idx],
                [warped_y_clamped, warped_x_clamped],
                order=1,
                mode='constant',
                cval=0
            )
            
            # Resample to target dimensions if needed
            if warped_band.shape != (target_height, target_width):
                from scipy.ndimage import zoom
                zoom_y = target_height / warped_band.shape[0]
                zoom_x = target_width / warped_band.shape[1]
                warped_band = zoom(warped_band, (zoom_y, zoom_x), order=1)
            
            # Crop to exact target size
            registered_data[band_idx] = warped_band[:target_height, :target_width].astype(source_data.dtype)


transformations not in memory, attempting to load from file...
❌ Could not load transformations: Expecting value: line 5 column 14 (char 75)
Please run Step 7 (Compute Transformation) first.


JSONDecodeError: Expecting value: line 5 column 14 (char 75)

## Step 9: Evaluate Accuracy Improvement

In [ ]:
# Check for required variables and set defaults if needed
try:
    _ = output_dir
except NameError:
    from pathlib import Path
    output_dir = Path("outputs")
    print(f"output_dir not defined, using default: {output_dir}")

try:
    _ = gcp_matching_dir
except NameError:
    gcp_matching_dir = output_dir / "gcp_matching"
    print(f"gcp_matching_dir not defined, using default: {gcp_matching_dir}")

try:
    _ = matches_dir
except NameError:
    matches_dir = gcp_matching_dir / "matches"
    matches_dir.mkdir(parents=True, exist_ok=True)
    print(f"matches_dir not defined, using default: {matches_dir}")

try:
    _ = registered_dir
except NameError:
    registered_dir = gcp_matching_dir / "registered"
    registered_dir.mkdir(parents=True, exist_ok=True)
    print(f"registered_dir not defined, using default: {registered_dir}")

try:
    _ = basemap_path
except NameError:
    from pathlib import Path
    data_dir = Path("/Users/mauriciohessflores/Documents/Code/Data/New Westminster Oct _25")
    basemap_path = data_dir / "Michael_RTK_orthos" / "TestsiteNewWest_Spexigeo_RTK.tiff"
    print(f"basemap_path not defined, using default: {basemap_path}")

try:
    _ = gcps
except NameError:
    print("⚠️  gcps not defined. Please run Step 2 first.")
    gcps = []

# Compare registered orthos to basemap

# Import required modules
from pathlib import Path
from typing import Dict, List, Optional, Tuple
import rasterio
import numpy as np

def evaluate_accuracy(ortho_path: Path, basemap_path: Path, gcps: List[Dict]) -> Dict:
    """
    Evaluate accuracy by comparing pixel values at GCP locations.
    """
    with rasterio.open(basemap_path) as basemap_src:
        basemap_data = basemap_src.read()
    
    with rasterio.open(ortho_path) as ortho_src:
        ortho_data = ortho_src.read()
    
    errors = []
    
    for gcp in gcps:
        pixel_coords = gcp_to_pixel_coords_from_utm(gcp['x_utm'], gcp['y_utm'], basemap_path)
        if not pixel_coords:
            continue
        
        col, row = pixel_coords
        
        if 0 <= row < basemap_data.shape[1] and 0 <= col < basemap_data.shape[2]:
            basemap_pixel = basemap_data[:, row, col]
            
            if 0 <= row < ortho_data.shape[1] and 0 <= col < ortho_data.shape[2]:
                ortho_pixel = ortho_data[:, row, col]
                
                # Compute error (Euclidean distance in pixel space)
                error = np.sqrt(np.sum((basemap_pixel.astype(float) - ortho_pixel.astype(float))**2))
                errors.append(error)
    
    if errors:
        return {
            'mean_error': float(np.mean(errors)),
            'rmse': float(np.sqrt(np.mean(np.array(errors)**2))),
            'max_error': float(np.max(errors)),
            'min_error': float(np.min(errors)),
            'num_points': len(errors)
        }
    else:
        return {
            'mean_error': 0.0,
            'rmse': 0.0,
            'max_error': 0.0,
            'min_error': 0.0,
            'num_points': 0
        }


# Evaluate accuracy for each registered ortho
print(f"\n{'='*60}")
print(f"Evaluating Accuracy Improvement")
print(f"{'='*60}")

# Check if registered_paths is defined
try:
    _ = registered_paths
except NameError:
    print("⚠️  registered_paths not defined. Please run Step 8 first.")
    registered_paths = {}

    # Try to load from transformations file or reconstruct from registered directory
    try:
        # Check if transformations file exists (from Step 7)
        transformations_file = matches_dir / "transformations.json"
        if transformations_file.exists():
            with open(transformations_file, 'r') as f:
                transformations = json.load(f)
            
            # Reconstruct registered_paths from transformations
            registered_paths = {}
            for ortho_name in transformations.keys():
                registered_path = registered_dir / f"{ortho_name}_registered.tif"
                if registered_path.exists():
                    registered_paths[ortho_name] = registered_path
            
            if len(registered_paths) > 0:
                print(f"✓ Reconstructed registered_paths from {transformations_file}")
                print(f"  Found {len(registered_paths)} registered orthos")
        else:
            # Try to find registered files directly
            if registered_dir.exists():
                registered_files = list(registered_dir.glob("*_registered.tif"))
                if registered_files:
                    registered_paths = {}
                    for reg_file in registered_files:
                        # Extract ortho name from filename (e.g., 'no_gcps_registered.tif' -> 'no_gcps')
                        ortho_name = reg_file.stem.replace('_registered', '')
                        registered_paths[ortho_name] = reg_file
                    
                    if len(registered_paths) > 0:
                        print(f"✓ Found {len(registered_paths)} registered orthos in {registered_dir}")
    except Exception as e:
        print(f"⚠️  Could not reconstruct registered_paths: {e}")

if len(registered_paths) == 0:
    print("⚠️  No registered orthos found. Please run Step 8 first.")
else:
    for ortho_name in registered_paths.keys():
        print(f"\nEvaluating {ortho_name}...")
        
        registered_path = registered_paths[ortho_name]
        
        if not registered_path.exists():
            print(f"  ⚠️  Registered ortho not found: {registered_path}")
            continue
        
        # Evaluate accuracy
        try:
            accuracy_metrics = evaluate_accuracy(
                registered_path,
                basemap_path,
                gcps
            )
            
            print(f"\n  Accuracy Metrics:")
            mean_err = accuracy_metrics.get('mean_error', 'N/A')
            if isinstance(mean_err, (int, float)):
                print(f"    Mean Error: {mean_err:.3f} m")
            else:
                print(f"    Mean Error: {mean_err}")
            
            rmse = accuracy_metrics.get('rmse', 'N/A')
            if isinstance(rmse, (int, float)):
                print(f"    RMSE: {rmse:.3f} m")
            else:
                print(f"    RMSE: {rmse}")
            
            max_err = accuracy_metrics.get('max_error', 'N/A')
            if isinstance(max_err, (int, float)):
                print(f"    Max Error: {max_err:.3f} m")
            else:
                print(f"    Max Error: {max_err}")
            
            print(f"    Points Evaluated: {accuracy_metrics.get('num_points', 0)}")

# Generate LaTeX report
latex_report_path = output_dir / "gcp_matching" / "accuracy_report.tex"
latex_report_path.parent.mkdir(parents=True, exist_ok=True)

latex_content = []
latex_content.append("\\documentclass[11pt]{article}")
latex_content.append("\\usepackage[utf8]{inputenc}")
latex_content.append("\\usepackage{graphicx}")
latex_content.append("\\usepackage{geometry}")
latex_content.append("\\geometry{a4paper, margin=1in}")
latex_content.append("\\usepackage{booktabs}")
latex_content.append("\\usepackage{float}")
latex_content.append("\\usepackage{caption}")
latex_content.append("\\begin{document}")
latex_content.append("\\title{GCP Matching Accuracy Evaluation Report}")
latex_content.append("\\author{Automated Analysis}")
latex_content.append("\\date{\\today}")
latex_content.append("\\maketitle")
latex_content.append("\\section{Executive Summary}")
latex_content.append("This report presents the accuracy evaluation of registered orthomosaics against the ground control basemap.")
latex_content.append("\\section{Accuracy Metrics}")
latex_content.append("\\begin{table}[H]")
latex_content.append("\\centering")
latex_content.append("\\begin{tabular}{lcccc}")
latex_content.append("\\toprule")
latex_content.append("Orthomosaic & Mean Error (m) & RMSE (m) & Max Error (m) & Points \\\\")
latex_content.append("\\midrule")

# Collect all accuracy metrics
all_accuracy_metrics = {}

for ortho_name in registered_paths.keys():
    registered_path = registered_paths[ortho_name]
    if not registered_path.exists():
        continue
    
    try:
        accuracy_metrics = evaluate_accuracy(registered_path, basemap_path, gcps)
        all_accuracy_metrics[ortho_name] = accuracy_metrics
        
        # Add to LaTeX table
        mean_err = accuracy_metrics.get('mean_error', 0.0)
        rmse = accuracy_metrics.get('rmse', 0.0)
        max_err = accuracy_metrics.get('max_error', 0.0)
        num_pts = accuracy_metrics.get('num_points', 0)
        
        mean_str = f"{mean_err:.3f}" if isinstance(mean_err, (int, float)) else "N/A"
        rmse_str = f"{rmse:.3f}" if isinstance(rmse, (int, float)) else "N/A"
        max_str = f"{max_err:.3f}" if isinstance(max_err, (int, float)) else "N/A"
        
        ortho_display = ortho_name.replace('_', ' ').title()
        latex_content.append(f"{ortho_display} & {mean_str} & {rmse_str} & {max_str} & {num_pts} \\\\")
    except Exception as e:
        print(f"  ⚠️  Error evaluating {ortho_name}: {e}")
        continue

latex_content.append("\\bottomrule")
latex_content.append("\\end{tabular}")
latex_content.append("\\caption{Accuracy metrics for registered orthomosaics}")
latex_content.append("\\label{tab:accuracy}")
latex_content.append("\\end{table}")

# Add visualization section if figures exist
latex_content.append("\\section{Visualizations}")

# Check for visualization files
vis_dir = gcp_matching_dir / "visualizations"
if vis_dir.exists():
    vis_files = list(vis_dir.glob("*.png")) + list(vis_dir.glob("*.jpg"))
    for vis_file in sorted(vis_files):
        # Copy to report directory for LaTeX
        report_vis_dir = latex_report_path.parent / "figures"
        report_vis_dir.mkdir(exist_ok=True)
        import shutil
        dest_file = report_vis_dir / vis_file.name
        if not dest_file.exists():
            shutil.copy(vis_file, dest_file)
        
        # Add figure to LaTeX
        fig_name = vis_file.stem.replace('_', ' ').title()
        latex_content.append(f"\\begin{{figure}}[H]")
        latex_content.append(f"\\centering")
        latex_content.append(f"\\includegraphics[width=0.8\\textwidth]{{figures/{vis_file.name}}}")
        latex_content.append(f"\\caption{{{fig_name}}}")
        latex_content.append(f"\\label{{fig:{vis_file.stem}}}")
        latex_content.append(f"\\end{{figure}}")

latex_content.append("\\section{Conclusion}")
latex_content.append("The registered orthomosaics show improved alignment with the ground control basemap.")
latex_content.append("\\end{document}")

# Write LaTeX file
with open(latex_report_path, 'w') as f:
    f.write('\n'.join(latex_content))

print(f"\n✓ LaTeX report generated: {latex_report_path}")
print(f"  To compile: pdflatex {latex_report_path.name}")
        except Exception as e:
            print(f"  ❌ Error evaluating accuracy: {e}")
